In [12]:
import numpy as np
from sklearn import datasets
import pandas as pd


In [ ]:

music = pd.read_csv('dataset.csv')
print(music.head())



   Unnamed: 0                track_id  ... time_signature track_genre
0           0  5SuOikwiRyPMVoIQDJUgSV  ...              4    acoustic
1           1  4qPNDBW1i3p13qLCt0Ki3A  ...              4    acoustic
2           2  1iJBSr7s7jYXzM8EGcbK5b  ...              4    acoustic
3           3  6lfxq3CG4xtTiEg7opyCyx  ...              3    acoustic
4           4  5vjLSffimiIP26QG5WcN2K  ...              4    acoustic

[5 rows x 21 columns]
